## 7.1 작은 이미지 모아 놓은 데이터셋

### 7.1.1 CIFAR-10 다운로드
- 32 X 32 크기의 컬러(RGB) 이미지 6만개
- 1에서 10까지의 정수로 레이블
    * 0 : 비행기 / 1 : 자동차 / 2 : 새 / 3 : 고양이 / 4 : 사슴 / 5 : 강아지 / 6 : 개구리 / 7 : 말 / 8 : 배 / 9 : 트럭

In [1]:
from torchvision import datasets

In [2]:
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # 훈련데이터용 데이터 객체 만듦
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:14<00:00, 11704315.84it/s]


Extracting C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/cifar-10-python.tar.gz to C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/
Files already downloaded and verified


In [3]:
type(cifar10).__mro__

(torchvision.datasets.cifar.CIFAR10,
 torchvision.datasets.vision.VisionDataset,
 torch.utils.data.dataset.Dataset,
 typing.Generic,
 object)

### 7.1.2 데이터셋 클래스
- torch.utils.data.Dataset의 서브클래스
    * Dataset은 데이터를 직접 들고 있지는 않지만, __ len __, __getitem__을 통해 접근할 수 있다.
        - __ len__ : 데이터셋의 아이템 수 반환
        - __ getitem__ : 샘플과 레이블(정수 인덱스)로 이루어진 아이템 반환

In [4]:
len(cifar10)

50000

In [5]:
# dataset에 __getitem__메소드가 구현되어 있으므로 개별 아이템에 접근할 때 표준 서브스크립트에 해당하는 색인용 튜플과 리스트 사용 가능
# 자동차에 해당하는 정수값이 1인 PIL 형식의 이미지
img, label = cifar10[99]
img, label

(<PIL.Image.Image image mode=RGB size=32x32>, 1)

In [6]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
class_names[label]

'automobile'

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow(img)
plt.show()

### 7.1.3 데이터 변환
- matplotlib으로 출력하기 위헤 permute() 사용하여 C X H X W를 H X W X C 형태로 변환

In [7]:
# PIL 이미지 -> 텐서로 변환
from torchvision import transforms
dir(transforms)

['AugMix',
 'AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'ElasticTransform',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandAugment',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 'TrivialAugmentWide',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional_pil',
 '_functional_tensor',
 '_presets',
 'autoaugment',
 'functional',
 'transforms']

In [8]:
to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
img_t.shape # C X H X W

torch.Size([3, 32, 32])

In [9]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform = transforms.ToTensor())

In [10]:
img_t, _ = tensor_cifar10[99]
type(img_t) # PIL 타입이 아닌 tensor로 리턴

torch.Tensor

In [11]:
img_t.shape, img_t.dtype

(torch.Size([3, 32, 32]), torch.float32)

In [12]:
img_t.min(), img_t.max() # PIL이미지는 0 ~ 255(채널당 8비트) 범위, ToTensor 변환으로 데이터가 채널당 32비트 부동소수점 형태가 되면서 0.0에서 1.0 사이로 범위 줄어듦

(tensor(0.), tensor(1.))

### 7.1.4 데이터 정규화
- -1 ~ +1 혹은 -2 ~ +2 사이에서 선형인 활성 함수 택하고 데이터를 같은 범위에서 평균을 가지게 한다면 뉴런은 0이 아닌 기울기를 가지게 되므로 빨리 학습할 수 있다.
- 각 채널을 정규화해 동일한 분산을 가지게 된다면, 채널 정보가 동일한 학습률로 경사 하강을 통해 섞인다.
- transforms.Normalize ; mean, stdev 계산하지 않음

In [18]:
import torch
import numpy as np
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [48]:
mean = imgs.view(3, -1).mean(dim=1)
mean_t = tuple(np.round(mean.tolist(),4))
mean, mean_t

(tensor([0.4914, 0.4822, 0.4465]), (0.4914, 0.4822, 0.4465))

In [20]:
imgs.view(3, -1).shape 

torch.Size([3, 51200000])

In [49]:
std = imgs.view(3, -1).std(dim=1)
std_t = tuple(np.round(std.tolist(),4))
std, std_t

(tensor([0.2470, 0.2435, 0.2616]), (0.247, 0.2435, 0.2616))

In [50]:
transforms.Normalize((mean_t), (std_t))

Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))

In [81]:
transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean_t),(std_t))
    ]))

In [111]:
transformed_cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean_t),(std_t))
    ]))

In [107]:
img_t, _ = transformed_cifar10[99]

## 7.2 새와 비행기 구별하기

### 7.2.1 데이터셋 구축

In [112]:
label_map = {0:0, 2:1} # 0 : 비행기, 2 : 새
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in transformed_cifar10 if label in [0,2]]
cifar2_val = [(img, label_map[label]) for img, label in transformed_cifar10_val if label in [0,2]]

In [113]:
len(cifar2), len(cifar2_val)

(10000, 2000)

### 7.2.2 완전 연결 모델

In [114]:
import torch.nn as nn

In [115]:
n_out = 2 # 비행기, 새

In [116]:
model = nn.Sequential(
        nn.Linear(3072, 512,), # 32 * 32 * 3 = 3072
        nn.Tanh(),
        nn.Linear(512, n_out))

### 7.2.3 분류기의 출력
- 소프트맥스
    * 출력값의 요소가 가질 수 있는 값은 [0.0, 1.0] 범위로 제한된다   
    -> 확률은 0보다 작거나 1보다 클 수 없으므로
    * 모든 출력 요소의 값의 합은 1.0이다.  
    -> 결과는 항상 새 아니면 비행기라고 가정
### 7.2.4 출력을 확률로 표현하기

In [89]:
def softmax(x) :
    return torch.exp(x) / torch.exp(x).sum()

In [90]:
softmax = nn.Softmax(dim = 1)

In [117]:
model = nn.Sequential(
        nn.Linear(3072, 512),
        nn.Tanh(),
        nn.Linear(512, 2),
        nn.Softmax(dim=1))

In [122]:
img, _ = cifar2[0]
img.shape, _ # label = 1

(torch.Size([3, 32, 32]), 1)

In [126]:
# 입력이 3072개의 피처를 가지고 있으며 0번 차원을 따라 배치로 이뤄지는 데이터
# 3 X 32 X 32 이미지를 1차원 텐서로 만들고 추가 차원을 0번 포지션에 넣는다.
img_batch = img.view(-1).unsqueeze(0)
img_batch.shape

torch.Size([1, 3072])

In [131]:
out = model(img_batch)
out # 선형계층의 가중치, 편향값 훈련되지 않고, 랜덤 값으로 초기화

tensor([[0.5953, 0.4047]], grad_fn=<SoftmaxBackward0>)

In [132]:
# argmax : 제일 높은 확률에 대한 인덱스 -> 레이블 구함
_, index = torch.max(out, dim=1)
index # 틀림

tensor([0])

### 7.2.5 분류를 위한 손실값
- 정답 클래스 out[class_index]와 관련된 확률 극대화  
-> 가능도 :정답 클래스에 대한 확률 수치  
    * 가능도 낮을 때, 즉 다른 클래스의 확률이 매우 높을 때 값이 커지는 손실 함수 필요
    * 가능도가 다른 클래스보다 높으면 손실값은 낮아야 한다.  
=> NLL(Negative Log Likelihood) = -sum(log(out_i[c_i]))
    

In [133]:
# 로그 확률의 텐서를 받는 nn.NLLLoss -> 입력을 확률의 로그값으로 받으면 확률이 0에 가까울 경우 문제
# => nn.Softmax() 대신 nn.LogSoftmax() 사용
model = nn.Sequential(
        nn.Linear(3072, 512),
        nn.Tanh(),
        nn.Linear(512, 2),
        nn.LogSoftmax(dim=1))

loss = nn.NLLLoss()

In [134]:
img, label = cifar2[0]
out = model(img.view(-1).unsqueeze(0))
loss(out, torch.tensor([label]))

tensor(0.9674, grad_fn=<NllLossBackward0>)

### 7.2.6 분류기 훈련

In [137]:
import torch.optim as optim

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
loss_fn = nn.NLLLoss()

n_epochs=100

for epoch in range(1, n_epochs+1) :
    for img, label in cifar2 :  # 한번에 하나의 샘플 평가
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
# 들쭉날쭉한 loss . . .

Epoch: 1, Loss: 6.434373
Epoch: 2, Loss: 17.315269
Epoch: 3, Loss: 6.000750
Epoch: 4, Loss: 6.839866
Epoch: 5, Loss: 4.823521
Epoch: 6, Loss: 9.278174
Epoch: 7, Loss: 16.225620
Epoch: 8, Loss: 2.514620
Epoch: 9, Loss: 0.002799
Epoch: 10, Loss: 7.355705
Epoch: 11, Loss: 4.979154
Epoch: 12, Loss: 15.045765
Epoch: 13, Loss: 7.951532
Epoch: 14, Loss: 7.577113
Epoch: 15, Loss: 15.258724
Epoch: 16, Loss: 12.777469
Epoch: 17, Loss: 10.226240
Epoch: 18, Loss: 8.091814
Epoch: 19, Loss: 12.951021
Epoch: 20, Loss: 4.535486
Epoch: 21, Loss: 13.321553
Epoch: 22, Loss: 4.383187
Epoch: 23, Loss: 7.040248
Epoch: 24, Loss: 1.412455
Epoch: 25, Loss: 4.507382
Epoch: 26, Loss: 7.903306
Epoch: 27, Loss: 11.454595
Epoch: 28, Loss: 0.782271
Epoch: 29, Loss: 0.091054
Epoch: 30, Loss: 2.631381
Epoch: 31, Loss: 4.315165
Epoch: 32, Loss: 0.000543
Epoch: 33, Loss: 3.160376
Epoch: 34, Loss: 10.463407
Epoch: 35, Loss: 4.504496
Epoch: 36, Loss: 2.021827
Epoch: 37, Loss: 5.648680
Epoch: 38, Loss: 9.216259
Epoch: 39, 

각 에포크마다 샘플을 섞은 후 한 번에 혹은 여러 개의 샘플에 대해 기울기 평가하면 경사 하강에 랜덤한 효과 줄 수 있다.  

  
torch.utils.data 모듈에는 미니 배치의 데이터를 섞거나 구조화하는 작업을 돕는 DataLoader 클래스 있다.   
-> 미니 배치에 포함될 샘플 가져올 때, 각 에포크마다 데이터 섞은 후 고르게 샘플링함

In [142]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

In [155]:
model = nn.Sequential(
        nn.Linear(3072, 512),
        nn.Tanh(),
        nn.Linear(512, 2),
        nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 100

for epoch in range(1, n_epochs + 1) :
    for imgs, labels in train_loader : # imgs.shape : 64 X 3 X 32 X 32
        batch_size = imgs.shape[0] # 64
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    
# 점점 떨어지는 loss . . . !

Epoch: 1, Loss: 0.373018
Epoch: 2, Loss: 0.408726
Epoch: 3, Loss: 0.357243
Epoch: 4, Loss: 0.305634
Epoch: 5, Loss: 0.187234
Epoch: 6, Loss: 0.785617
Epoch: 7, Loss: 0.814715
Epoch: 8, Loss: 0.425080
Epoch: 9, Loss: 0.289232
Epoch: 10, Loss: 0.322380
Epoch: 11, Loss: 0.491398
Epoch: 12, Loss: 0.453093
Epoch: 13, Loss: 0.412017
Epoch: 14, Loss: 0.247110
Epoch: 15, Loss: 0.497091
Epoch: 16, Loss: 0.724647
Epoch: 17, Loss: 0.315986
Epoch: 18, Loss: 0.470300
Epoch: 19, Loss: 0.172258
Epoch: 20, Loss: 0.177786
Epoch: 21, Loss: 0.230062
Epoch: 22, Loss: 0.306673
Epoch: 23, Loss: 0.302494
Epoch: 24, Loss: 0.422032
Epoch: 25, Loss: 0.134306
Epoch: 26, Loss: 0.266599
Epoch: 27, Loss: 0.165318
Epoch: 28, Loss: 0.381498
Epoch: 29, Loss: 0.336131
Epoch: 30, Loss: 0.112090
Epoch: 31, Loss: 0.109310
Epoch: 32, Loss: 0.297195
Epoch: 33, Loss: 0.266919
Epoch: 34, Loss: 0.160415
Epoch: 35, Loss: 0.152047
Epoch: 36, Loss: 0.080195
Epoch: 37, Loss: 0.067546
Epoch: 38, Loss: 0.079039
Epoch: 39, Loss: 0.09

In [143]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=True)

In [144]:
correct = 0
total = 0

with torch.no_grad() :
    for imgs, labels in val_loader :
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0] # 64 * 31 + 16 = 2000
        correct += int((predicted == labels).sum()) # 맞춘 갯수

In [148]:
correct/total # 정확도

0.7535

nn.LogSoftmax() + nn.NLLLoss() = nn.CrossEntropyLoss()  
    * nn.NLLLoss() : 입력으로 로그 확률 예측 받음  
    * nn.CrossEntropyLoss() : 입력으로 점수 받음(logits)

In [156]:
model = nn.Sequential(
        nn.Linear(3072, 1024),
        nn.Tanh(),
        nn.Linear(1024, 512),
        nn.Tanh(),
        nn.Linear(512, 128),
        nn.Tanh(),
        nn.Linear(128, 2))

loss_fn = nn.CrossEntropyLoss()

In [157]:
# parameter 개수 -> requires_grad=True 여야함
numel_list = [p.numel() for p in model.parameters() if p.requires_grad==True]
sum(numel_list), numel_list  # (가중치, 편향) * 4개의 층

(3737474, [3145728, 1024, 524288, 512, 65536, 128, 256, 2])

In [158]:
numel_list1 = [p.numel() for p in nn.Linear(3072, 1024).parameters() if p.requires_grad==True]
sum(numel_list1), numel_list1

(3146752, [3145728, 1024])

In [160]:
numel_list2 = [p.numel() for p in nn.Linear(3072, 512).parameters() if p.requires_grad==True]
sum(numel_list2), numel_list2 # 파라미터 수 적음

(1573376, [1572864, 512])

In [162]:
linear = nn.Linear(3072, 1024)
linear.weight.shape, linear.bias.shape

(torch.Size([1024, 3072]), torch.Size([1024]))